In [12]:
import pickle

with open('genetic_algorithm_selected_features_per_method.pkl', 'rb') as f:
    selected_features = pickle.load(f)

selected_features

{'mrmr': Index([105,  64, 131, 136, 100, 102,  24, 103, 104, 359, 101,  29, 266,  57,
         39, 241, 351, 155, 335, 442],
       dtype='int64'),
 'surf': Index([105, 102, 101, 321, 100, 289, 283, 374, 254, 311,
        ...
        378, 228, 354, 394, 232, 332, 299, 279, 356, 382],
       dtype='int64', length=490),
 'forest': Index([100, 101, 102, 103, 104, 105, 403], dtype='int64'),
 'xgb': Index([100, 101, 102, 103, 104, 105], dtype='int64'),
 'common_features': [105.0,
  64.0,
  131.0,
  136.0,
  100.0,
  102.0,
  24.0,
  103.0,
  104.0,
  359.0,
  101.0,
  29.0,
  266.0,
  57.0,
  39.0,
  241.0,
  351.0,
  155.0,
  335.0,
  442.0,
  403.0]}

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

X = pd.read_table('./data/x_train.txt', header=None, delimiter=' ')
y = pd.read_table('./data/y_train.txt', header=None)

X_test = pd.read_table('./data/x_test.txt', header=None, delimiter=' ')

X.head()

correlated_features_to_remove = [1, 0, 4, 6, 7, 3, 5, 8, 9, 2]
X = X.drop(correlated_features_to_remove, axis=1)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations

common_features = selected_features['common_features']
common_features_sixes_permutations = (
    list(combinations(common_features, 4))
    + list(combinations(common_features, 3))
    + list(combinations(common_features, 2))
    + list(combinations(common_features, 1))
)

len(common_features_sixes_permutations)

7546

In [ ]:
from tqdm import tqdm
feature_selection_scores = []
current_max_score = 0
current_best_feature_selection = None

tqdm._instances.clear()
bar = tqdm(total=len(common_features_sixes_permutations))
for feature_selection in common_features_sixes_permutations:
    X_train_selected = X_train[pd.Index(feature_selection)]
    X_val_selected = X_val[pd.Index(feature_selection)]

    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(X_train_selected, y_train.values.ravel())
    score = clf.score(X_val_selected, y_val)
    feature_selection_scores.append((feature_selection, score))
    
    bar.update(1)
    current_max_score = max(current_max_score, score)
    current_best_feature_selection = feature_selection if score == current_max_score else current_best_feature_selection
    bar.set_description(f'Score: {score}, Max score: {current_max_score}, Best feature selection: {current_best_feature_selection}')


Score: 0.6, Max score: 0.662, Best feature selection: (105.0, 100.0, 102.0, 101.0):  12%|█▏        | 726/5985 [01:58<14:21,  6.11it/s]
Score: 0.48, Max score: 0.662, Best feature selection: (105.0, 100.0, 102.0, 101.0):  44%|████▍     | 3319/7546 [07:42<11:10,  6.30it/s] 

In [11]:
current_best_feature_selection

# (100.0, 101.0, 102.0, 105.0)
# + 103.0
# + 136.0
# + 104.0



(100.0, 101.0, 102.0, 103.0, 105.0)